## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now() - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=6)

def format_date(date):
    day = date.day
    # Months 3-7 (March–July) should be full; all others abbreviated with a period
    if date.month in (3, 4, 5, 6, 7):
        month_str = date.strftime("%B")      # e.g. "March", "April", ...
    else:
        month_str = date.strftime("%b") + "."  # e.g. "Jan.", "Feb.", "Aug.", ...

    return f"{month_str} {day}"


# Format both dates
week_ending = format_date(today)
week_starting = format_date(one_week_ago)

print(week_starting, week_ending)

June 30 July 6


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,June-30-2025,Condo/Co-op,17749 Collins Ave Unit TS2401/02,Sunny Isles Beach,FL,33160.0,13500000.0,4.0,5.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11825367,N,Y,25.940065,-80.120264


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df2 = df[(df['SOLD DATE'] >= one_week_ago) & (df['SOLD DATE'] < today)]

In [10]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-07-01    29
2025-07-02    21
2025-07-03    16
2025-07-04     1
2025-07-06     1
Name: count, dtype: int64

In [11]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [12]:
df_top_ten = df2.head(10)

In [13]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
62,PAST SALE,2025-07-01,Condo/Co-op,2627 S Bayshore Dr #2304,Coconut Grove,FL,33133.0,4925000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11752482,N,Y,25.732272,-80.234921
53,PAST SALE,2025-07-03,Condo/Co-op,1201 20th St Ph 02,Miami Beach,FL,33139.0,3500000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1201-20t...,MARMLS,A11748694,N,Y,25.795698,-80.142055
32,PAST SALE,2025-07-03,Condo/Co-op,718 Valencia Ave Ph 505,Coral Gables,FL,33134.0,2815000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/718-Val...,MARMLS,A11777513,N,Y,25.746877,-80.269580
2,PAST SALE,2025-07-01,Condo/Co-op,9801 Collins Ave Unit 17S,Bal Harbour,FL,33154.0,2650000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9801-Col...,MARMLS,A11786518,N,Y,25.890231,-80.122819
45,PAST SALE,2025-07-02,Condo/Co-op,17555 Collins Ave #2903,Sunny Isles Beach,FL,33160.0,2050000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11772124,N,Y,25.939167,-80.120677
35,PAST SALE,2025-07-02,Condo/Co-op,1221 SW 12 Ct,Miami,FL,33135.0,1815000.0,4.0,5.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1221-SW-12th-C...,MARMLS,A11754226,N,Y,25.761555,-80.214980
96,PAST SALE,2025-07-01,Condo/Co-op,11 Island Ave #1512,Miami Beach,FL,33139.0,1750000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/11-Islan...,MARMLS,A11708184,N,Y,25.789357,-80.147840
86,PAST SALE,2025-07-06,Condo/Co-op,300 Sunny Isle #808,Sunny Isles Beach,FL,33160.0,1500000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/30...,MARMLS,A11751885,N,Y,25.929383,-80.125611
84,PAST SALE,2025-07-01,Condo/Co-op,3131 NE 7th Ave #706,Miami,FL,33137.0,1425000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3131-NE-7th-Av...,MARMLS,A11735423,N,Y,25.807047,-80.185578
77,PAST SALE,2025-07-01,Condo/Co-op,20201 E Country Club Dr #1502,Aventura,FL,33180.0,1325000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/20201-E-Countr...,MARMLS,A11754067,N,Y,25.964660,-80.125413


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [14]:
def process_response(url):
    try:
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"
        querystring = {"url": url}
        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)
        time.sleep(0.2)
        response.raise_for_status()

        # If .json() returns None, fall back on empty dict
        data = response.json() or {}

        # Drill down with or {} at each step to avoid None.get errors
        panel_info = (data.get('data') or {}) \
            .get('mainHouseInfoPanelInfo') or {}
        main_info = panel_info.get('mainHouseInfo') or {}

        # Now safe to pull lists
        listing_agents = main_info.get('listingAgents') or []
        buying_agents  = main_info.get('buyingAgents')  or []

        # Defaults
        list_agent_name_1 = list_broker_name_1 = None
        list_agent_name_2 = list_broker_name_2 = None
        buy_agent_name_1  = buy_broker_name_1  = None
        buy_agent_name_2  = buy_broker_name_2  = None

        # Extract up to two listing agents
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}) \
                                             .get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}) \
                                             .get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')

        # Extract up to two buying agents
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}) \
                                            .get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}) \
                                            .get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')

        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }


In [15]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [16]:
response_df = pd.DataFrame(response_list)

In [17]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [18]:
df_top_ten = merged_df

## Current Week's Values

In [19]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

68


In [20]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$700,760


In [21]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$495


In [22]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$47,651,700


In [23]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [24]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input county name:


 Miami-Dade


Input Previous Week Condo Sales Total (number units sold):
84
-----------
Input Previous Week Condo Average Sales Price:
715219
-----------
Input Previous Week Condo Average PSF:
547
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
60078400.0


In [25]:
last_week

,Condo Sales,Average Sale Price,Average PSF,Last Week Total Dollar Volume
0,84,715219,547,60078400.0


In [26]:
last_week['Last Week Total Dollar Volume'].iloc[0]

60078400.0

In [27]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0           68              700760          495                     47651700.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [28]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [29]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [30]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [31]:
pd.set_option('display.max_columns',None)

In [32]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-07-01,Condo/Co-op,2627 S Bayshore Dr #2304,Coconut Grove,FL,33133.0,4925000.0,3.0,3.5,Grovenor House,2640.0,NaN,2005.0,NaN,1866.0,4081.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11752482,N,Y,25.732272,-80.234921,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,Lucas Boccheciampe,Vantage Real Estate LLC,None,None,Debora Caridad,Fortune Christie's International Real Estate,None,None,1,orange


In [33]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [34]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-07-01,Condo/Co-op,2627 S Bayshore Dr #2304,Coconut Grove,FL,33133.0,4925000.0,3.0,3.5,Grovenor House,2640.0,NaN,2005.0,NaN,1866.0,4081.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11752482,N,Y,25.732272,-80.234921,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,Lucas Boccheciampe,Vantage Real Estate LLC,None,None,Debora Caridad,Fortune Christie's International Real Estate,None,None,1,orange
1,PAST SALE,2025-07-03,Condo/Co-op,1201 20th St Ph 02,Miami Beach,FL,33139.0,3500000.0,3.0,3.5,ISLAND VIEW ADDN,2031.0,NaN,2016.0,NaN,1723.0,3993.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1201-20t...,MARMLS,A11748694,N,Y,25.795698,-80.142055,https://www.redfin.com/FL/Miami-Beach/1201-20t...,Liz Hogan,"Compass Florida, LLC.",Kaitlyn Neumann,"Compass Florida, LLC",Liz Hogan,"Compass Florida, LLC.",Kaitlyn Neumann,"Compass Florida, LLC",2,blue
2,PAST SALE,2025-07-03,Condo/Co-op,718 Valencia Ave Ph 505,Coral Gables,FL,33134.0,2815000.0,3.0,3.5,BILTMORE PARC CONDO,2345.0,NaN,2017.0,NaN,1200.0,2700.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/718-Val...,MARMLS,A11777513,N,Y,25.746877,-80.269580,https://www.redfin.com/FL/Coral-Gables/718-Val...,Anniella Tabraue,One Sotheby's International Realty,None,None,Tesalia Sacasa,BHHS EWM Realty,None,None,3,blue
3,PAST SALE,2025-07-01,Condo/Co-op,9801 Collins Ave Unit 17S,Bal Harbour,FL,33154.0,2650000.0,3.0,3.0,BALMORAL CONDO,1942.0,NaN,1977.0,NaN,1365.0,2451.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9801-Col...,MARMLS,A11786518,N,Y,25.890231,-80.122819,https://www.redfin.com/FL/Bal-Harbour/9801-Col...,Eva Bouhadana,Beachfront Realty Inc,Nicole Bouhadana,Beachfront Realty Inc,Jacob Assouline,"Luxuri International Real Estate Miami, LLC.",None,None,4,blue
4,PAST SALE,2025-07-02,Condo/Co-op,17555 Collins Ave #2903,Sunny Isles Beach,FL,33160.0,2050000.0,3.0,3.0,THE PINNACLE CONDO,1840.0,NaN,1998.0,NaN,1114.0,2130.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11772124,N,Y,25.939167,-80.120677,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Ana Waxman,Keller Williams Eagle Realty,Evgeniy Binev,Keller Williams Eagle Realty,Michael Lombard,Coldwell Banker Realty,None,None,5,blue
5,PAST SALE,2025-07-02,Condo/Co-op,1221 SW 12 Ct,Miami,FL,33135.0,1815000.0,4.0,5.0,SECOND WESTMORELAND ADD,3103.0,NaN,2025.0,NaN,585.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1221-SW-12th-C...,MARMLS,A11754226,N,Y,25.761555,-80.214980,https://www.redfin.com/FL/Miami/1221-SW-12th-C...,Nancy Muniz,Coldwell Banker Realty,None,None,Lina Barcelo,Douglas Elliman,None,None,6,blue
6,PAST SALE,2025-07-01,Condo/Co-op,11 Island Ave #1512,Miami Beach,FL,33139.0,1750000.0,2.0,2.5,COSTA BRAVA CONDO,1733.0,NaN,1972.0,NaN,1010.0,2093.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/11-Islan...,MARMLS,A11708184,N,Y,25.789357,-80.147840,https://www.redfin.com/FL/Miami-Beach/11-Islan...,Stefania Cambarau,Douglas Elliman,David Siddons,Douglas Elliman,Yaron Lavi,Sterling Equity Realty LLC,None,None,7,blue
7,PAST SALE,2025-07-06,Condo/Co-op,300 Sunny Isle #808,Sunny Isles Beach,FL,33160.0,1500000.0,3.0,3.5,Parque Towers,1892.0,NaN,2019.0,NaN,793.0,1695.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/30...,MARMLS,A11751885,N,Y,25.929383,-80.125611,https://www.redfin.com/FL/Sunny-Isles-Beach/30...,Lana Bell,Douglas Elliman,None,None,Alena Siadletskaya,Vera Realty LLC,None,None,8,blue
8,PAST SALE,2025-07-01,Condo/Co-op,3131 NE 7th Ave #706,Miami,FL,33137.0,1425000.0,3.0,3.5,ONE PARAISO C

In [35]:
%store -r map_box_api_key

In [36]:
import folium
from branca.element import Template, MacroElement

# ——— Your title & caption HTML ———
title_html = f'''
  <h3 align="center" style="font-size:16px"><b>Recent {county} County Condo Sales</b></h3>
'''
caption_html = f'''
  <p align="center" style="font-size:13px"><i>{week_starting} — {week_ending}</i></p>
'''

# ——— Base map ———
m = folium.Map(
    location=df[['LATITUDE','LONGITUDE']].mean().to_list(),
    zoom_start=10.5,
    tiles=None
)

# ——— Tile layers ———
folium.TileLayer('OpenStreetMap', name='OSM', control=False).add_to(m)
folium.TileLayer(
    tiles=(
        "https://api.mapbox.com/styles/v1/"
        "mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox", name="Streets", control=False
).add_to(m)

# ——— Add your markers ———
for _, row in df_top_ten.iterrows():
    lat, lon = row['LATITUDE'], row['LONGITUDE']
    popup = folium.Popup(popup_html(row), max_width=400)

    if row['COLOR'] == 'blue':
        # Regular sale: blue circle
        folium.CircleMarker(
            location=(lat, lon),
            radius=7,
            fill=True,
            fill_color='blue',
            fill_opacity=0.5,
            color=None,
            popup=popup
        ).add_to(m)
    else:
        # Top sale: gold star
        folium.Marker(
            location=(lat, lon),
            popup=popup,
            icon=folium.DivIcon(
                html="""
                <div style="font-size:24px; line-height:24px; color:gold;">
                  &#9733;
                </div>
                """
            )
        ).add_to(m)

# ——— Layer control, title & caption ———
folium.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

# ——— Optional legend via branca ———
legend_html = """
{% macro html(this, kwargs) %}
  <div style="
      position: fixed;
      bottom: 50px; left: 50px;
      background: white; padding: 8px;
      border: 2px solid grey;
      font-size: 12px;
      z-index: 9999;
  ">
    <div style="display:inline-block; font-size:18px; color:gold; margin-right:6px;">&#9733;</div> Top Sale<br>
    <i style="background: blue; width: 10px; height: 10px;
       display: inline-block; margin-right: 6px;"></i> All other sales
  </div>
{% endmacro %}
"""
legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)

# ——— Display ———
m


In [37]:
m.save('index.html')

## Data snagger

In [38]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [39]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-07-01,Condo/Co-op,2627 S Bayshore Dr #2304,Coconut Grove,FL,33133.0,4925000.0,3.0,3.5,Grovenor House,2640.0,NaN,2005.0,NaN,1866.0,4081.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11752482,N,Y,25.732272,-80.234921,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,Lucas Boccheciampe,Vantage Real Estate LLC,None,None,Debora Caridad,Fortune Christie's International Real Estate,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [40]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_07072025


# CREATE TEMPLATE 

In [41]:
muni_set = set(df_top_ten['CITY'])

In [42]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [43]:
df_top_ten.reset_index(inplace=True,drop=True)

In [44]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [45]:
top_sale

'Grovenor House at 2627 S Bayshore Dr #2304 in Coconut Grove'

In [46]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [47]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [48]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-07-01,Condo/Co-op,2627 S Bayshore Dr #2304,Coconut Grove,FL,33133.0,4925000.0,3.0,3.5,Grovenor House,2640.0,NaN,2005.0,NaN,1866.0,4081.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11752482,N,Y,25.732272,-80.234921,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,Lucas Boccheciampe,Vantage Real Estate LLC,None,None,Debora Caridad,Fortune Christie's International Real Estate,None,None,1,orange,Grovenor House at 2627 S Bayshore Dr #2304 in ...
1,PAST SALE,2025-07-03,Condo/Co-op,1201 20th St Ph 02,Miami Beach,FL,33139.0,3500000.0,3.0,3.5,ISLAND VIEW ADDN,2031.0,NaN,2016.0,NaN,1723.0,3993.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1201-20t...,MARMLS,A11748694,N,Y,25.795698,-80.142055,https://www.redfin.com/FL/Miami-Beach/1201-20t...,Liz Hogan,"Compass Florida, LLC.",Kaitlyn Neumann,"Compass Florida, LLC",Liz Hogan,"Compass Florida, LLC.",Kaitlyn Neumann,"Compass Florida, LLC",2,blue,ISLAND VIEW ADDN at 1201 20th St Ph 02 in Miam...
2,PAST SALE,2025-07-03,Condo/Co-op,718 Valencia Ave Ph 505,Coral Gables,FL,33134.0,2815000.0,3.0,3.5,BILTMORE PARC CONDO,2345.0,NaN,2017.0,NaN,1200.0,2700.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/718-Val...,MARMLS,A11777513,N,Y,25.746877,-80.269580,https://www.redfin.com/FL/Coral-Gables/718-Val...,Anniella Tabraue,One Sotheby's International Realty,None,None,Tesalia Sacasa,BHHS EWM Realty,None,None,3,blue,BILTMORE PARC CONDO at 718 Valencia Ave Ph 505...
3,PAST SALE,2025-07-01,Condo/Co-op,9801 Collins Ave Unit 17S,Bal Harbour,FL,33154.0,2650000.0,3.0,3.0,BALMORAL CONDO,1942.0,NaN,1977.0,NaN,1365.0,2451.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9801-Col...,MARMLS,A11786518,N,Y,25.890231,-80.122819,https://www.redfin.com/FL/Bal-Harbour/9801-Col...,Eva Bouhadana,Beachfront Realty Inc,Nicole Bouhadana,Beachfront Realty Inc,Jacob Assouline,"Luxuri International Real Estate Miami, LLC.",None,None,4,blue,BALMORAL CONDO at 9801 Collins Ave Unit 17S in...
4,PAST SALE,2025-07-02,Condo/Co-op,17555 Collins Ave #2903,Sunny Isles Beach,FL,33160.0,2050000.0,3.0,3.0,THE PINNACLE CONDO,1840.0,NaN,1998.0,NaN,1114.0,2130.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11772124,N,Y,25.939167,-80.120677,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Ana Waxman,Keller Williams Eagle Realty,Evgeniy Binev,Keller Williams Eagle Realty,Michael Lombard,Coldwell Banker Realty,None,None,5,blue,THE PINNACLE CONDO at 17555 Collins Ave #2903 ...
5,PAST SALE,2025-07-02,Condo/Co-op,1221 SW 12 Ct,Miami,FL,33135.0,1815000.0,4.0,5.0,SECOND WESTMORELAND ADD,3103.0,NaN,2025.0,NaN,585.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1221-SW-12th-C...,MARMLS,A11754226,N,Y,25.761555,-80.214980,https://www.redfin.com/FL/Miami/1221-SW-12th-C...,Nancy Muniz,Coldwell Banker Realty,None,None,Lina Barcelo,Douglas Elliman,None,None,6,blue,SECOND WESTMORELAND ADD at 1221 SW 12 Ct in Miami
6,PAST SALE,2025-07-01,Condo/Co-op,11 Island Ave #1512,Miami Beach,FL,33139.0,1750000.0,2.0,2.5,COSTA BRAVA CONDO,1733.0,NaN,1972.0,NaN,1010.0,2093.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/11-Islan...,MARMLS,A11708184,N,Y,25.789357,-80.147840,https://www.redfin.com/FL/Miami-Beach/11-Islan...,Stefania Cambarau,Douglas Elliman,David Siddons,Douglas Elliman,Yaron Lavi,Sterling Equity Realty LLC,None,None,7,blue,COSTA BRAVA CONDO at 11 Island Ave #1512 in Mi...
7,PAST SALE,2025-07-06,Condo/Co-op,300 Sunny Isle #808,Sunny Isles Beach,FL,33160.0,1500000.0,3.0,3.5,Parque Towers,1892.0,NaN,2019.0,NaN,

In [49]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Grovenor House closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,325,000 to $4,925,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami, Aventura, Bal Harbour, Coconut Grove, Sunny Isles Beach, Miami Beach, Coral Gables, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 68 condo sales totaling $47,651,700 million from June 30 to July 6. The previous week, brokers closed 84 condo sales totaling $60,078,400.

Last week’s units sold for an average of $700,760, lower than the $715,2

In [50]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [51]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [52]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [53]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-07-01,Condo/Co-op,2627 S Bayshore Dr #2304,Coconut Grove,FL,33133.0,4925000.0,3.0,3.5,Grovenor House,2640.0,NaN,2005.0,NaN,1866.0,4081.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11752482,N,Y,25.732272,-80.234921,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,Lucas Boccheciampe,Vantage Real Estate LLC,None,None,Debora Caridad,Fortune Christie's International Real Estate,None,None,1,orange,Grovenor House at 2627 S Bayshore Dr #2304 in ...
1,PAST SALE,2025-07-03,Condo/Co-op,1201 20th St Ph 02,Miami Beach,FL,33139.0,3500000.0,3.0,3.5,ISLAND VIEW ADDN,2031.0,NaN,2016.0,NaN,1723.0,3993.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1201-20t...,MARMLS,A11748694,N,Y,25.795698,-80.142055,https://www.redfin.com/FL/Miami-Beach/1201-20t...,Liz Hogan,"Compass Florida, LLC.",Kaitlyn Neumann,"Compass Florida, LLC",Liz Hogan,"Compass Florida, LLC.",Kaitlyn Neumann,"Compass Florida, LLC",2,blue,ISLAND VIEW ADDN at 1201 20th St Ph 02 in Miam...
2,PAST SALE,2025-07-03,Condo/Co-op,718 Valencia Ave Ph 505,Coral Gables,FL,33134.0,2815000.0,3.0,3.5,BILTMORE PARC CONDO,2345.0,NaN,2017.0,NaN,1200.0,2700.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/718-Val...,MARMLS,A11777513,N,Y,25.746877,-80.269580,https://www.redfin.com/FL/Coral-Gables/718-Val...,Anniella Tabraue,One Sotheby's International Realty,None,None,Tesalia Sacasa,BHHS EWM Realty,None,None,3,blue,BILTMORE PARC CONDO at 718 Valencia Ave Ph 505...
3,PAST SALE,2025-07-01,Condo/Co-op,9801 Collins Ave Unit 17S,Bal Harbour,FL,33154.0,2650000.0,3.0,3.0,BALMORAL CONDO,1942.0,NaN,1977.0,NaN,1365.0,2451.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9801-Col...,MARMLS,A11786518,N,Y,25.890231,-80.122819,https://www.redfin.com/FL/Bal-Harbour/9801-Col...,Eva Bouhadana,Beachfront Realty Inc,Nicole Bouhadana,Beachfront Realty Inc,Jacob Assouline,"Luxuri International Real Estate Miami, LLC.",None,None,4,blue,BALMORAL CONDO at 9801 Collins Ave Unit 17S in...
4,PAST SALE,2025-07-02,Condo/Co-op,17555 Collins Ave #2903,Sunny Isles Beach,FL,33160.0,2050000.0,3.0,3.0,THE PINNACLE CONDO,1840.0,NaN,1998.0,NaN,1114.0,2130.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11772124,N,Y,25.939167,-80.120677,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Ana Waxman,Keller Williams Eagle Realty,Evgeniy Binev,Keller Williams Eagle Realty,Michael Lombard,Coldwell Banker Realty,None,None,5,blue,THE PINNACLE CONDO at 17555 Collins Ave #2903 ...
5,PAST SALE,2025-07-02,Condo/Co-op,1221 SW 12 Ct,Miami,FL,33135.0,1815000.0,4.0,5.0,SECOND WESTMORELAND ADD,3103.0,NaN,2025.0,NaN,585.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1221-SW-12th-C...,MARMLS,A11754226,N,Y,25.761555,-80.214980,https://www.redfin.com/FL/Miami/1221-SW-12th-C...,Nancy Muniz,Coldwell Banker Realty,None,None,Lina Barcelo,Douglas Elliman,None,None,6,blue,SECOND WESTMORELAND ADD at 1221 SW 12 Ct in Miami
6,PAST SALE,2025-07-01,Condo/Co-op,11 Island Ave #1512,Miami Beach,FL,33139.0,1750000.0,2.0,2.5,COSTA BRAVA CONDO,1733.0,NaN,1972.0,NaN,1010.0,2093.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/11-Islan...,MARMLS,A11708184,N,Y,25.789357,-80.147840,https://www.redfin.com/FL/Miami-Beach/11-Islan...,Stefania Cambarau,Douglas Elliman,David Siddons,Douglas Elliman,Yaron Lavi,Sterling Equity Realty LLC,None,None,7,blue,COSTA BRAVA CONDO at 11 Island Ave #1512 in Mi...
7,PAST SALE,2025-07-06,Condo/Co-op,300 Sunny Isle #808,Sunny Isles Beach,FL,33160.0,1500000.0,3.0,3.5,Parque Towers,1892.0,NaN,2019.0,NaN,

In [54]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Grovenor House closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,325,000 to $4,925,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami, Aventura, Bal Harbour, Coconut Grove, Sunny Isles Beach, Miami Beach, Coral Gables, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 68 condo sales totaling $47,651,700 million from June 30 to July 6. The previous week, brokers closed 84 condo sales totaling $60,078,400.

Last week’s units sold for an average of $700,760, lower than the $715,2

In [62]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Miami/2627-S-Bayshore-Dr-33133/unit-2304/home/43345020


In [63]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami-Beach/1201-20th-St-33139/unit-2/home/101703765


In [64]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami/20201-E-Country-Club-Dr-33180/unit-1502/home/43309701


In [65]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami/3131-NE-7th-Ave-33137/unit-706/home/79137008


In [66]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami/2627-S-Bayshore-Dr-33133/unit-2304/home/43345020


In [67]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Miami/20201-E-Country-Club-Dr-33180/unit-1502/home/43309701


In [68]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-07-01,Condo/Co-op,2627 S Bayshore Dr #2304,Coconut Grove,FL,33133.0,4925000.0,3.0,3.5,Grovenor House,2640.0,NaN,2005.0,NaN,1866.0,4081.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11752482,N,Y,25.732272,-80.234921,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,Lucas Boccheciampe,Vantage Real Estate LLC,None,None,Debora Caridad,Fortune Christie's International Real Estate,None,None,1,orange,Grovenor House at 2627 S Bayshore Dr #2304 in ...
1,PAST SALE,2025-07-03,Condo/Co-op,1201 20th St Ph 02,Miami Beach,FL,33139.0,3500000.0,3.0,3.5,ISLAND VIEW ADDN,2031.0,NaN,2016.0,NaN,1723.0,3993.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1201-20t...,MARMLS,A11748694,N,Y,25.795698,-80.142055,https://www.redfin.com/FL/Miami-Beach/1201-20t...,Liz Hogan,"Compass Florida, LLC.",Kaitlyn Neumann,"Compass Florida, LLC",Liz Hogan,"Compass Florida, LLC.",Kaitlyn Neumann,"Compass Florida, LLC",2,blue,ISLAND VIEW ADDN at 1201 20th St Ph 02 in Miam...
2,PAST SALE,2025-07-03,Condo/Co-op,718 Valencia Ave Ph 505,Coral Gables,FL,33134.0,2815000.0,3.0,3.5,BILTMORE PARC CONDO,2345.0,NaN,2017.0,NaN,1200.0,2700.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/718-Val...,MARMLS,A11777513,N,Y,25.746877,-80.269580,https://www.redfin.com/FL/Coral-Gables/718-Val...,Anniella Tabraue,One Sotheby's International Realty,None,None,Tesalia Sacasa,BHHS EWM Realty,None,None,3,blue,BILTMORE PARC CONDO at 718 Valencia Ave Ph 505...
3,PAST SALE,2025-07-01,Condo/Co-op,9801 Collins Ave Unit 17S,Bal Harbour,FL,33154.0,2650000.0,3.0,3.0,BALMORAL CONDO,1942.0,NaN,1977.0,NaN,1365.0,2451.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9801-Col...,MARMLS,A11786518,N,Y,25.890231,-80.122819,https://www.redfin.com/FL/Bal-Harbour/9801-Col...,Eva Bouhadana,Beachfront Realty Inc,Nicole Bouhadana,Beachfront Realty Inc,Jacob Assouline,"Luxuri International Real Estate Miami, LLC.",None,None,4,blue,BALMORAL CONDO at 9801 Collins Ave Unit 17S in...
4,PAST SALE,2025-07-02,Condo/Co-op,17555 Collins Ave #2903,Sunny Isles Beach,FL,33160.0,2050000.0,3.0,3.0,THE PINNACLE CONDO,1840.0,NaN,1998.0,NaN,1114.0,2130.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11772124,N,Y,25.939167,-80.120677,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Ana Waxman,Keller Williams Eagle Realty,Evgeniy Binev,Keller Williams Eagle Realty,Michael Lombard,Coldwell Banker Realty,None,None,5,blue,THE PINNACLE CONDO at 17555 Collins Ave #2903 ...
5,PAST SALE,2025-07-02,Condo/Co-op,1221 SW 12 Ct,Miami,FL,33135.0,1815000.0,4.0,5.0,SECOND WESTMORELAND ADD,3103.0,NaN,2025.0,NaN,585.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1221-SW-12th-C...,MARMLS,A11754226,N,Y,25.761555,-80.214980,https://www.redfin.com/FL/Miami/1221-SW-12th-C...,Nancy Muniz,Coldwell Banker Realty,None,None,Lina Barcelo,Douglas Elliman,None,None,6,blue,SECOND WESTMORELAND ADD at 1221 SW 12 Ct in Miami
6,PAST SALE,2025-07-01,Condo/Co-op,11 Island Ave #1512,Miami Beach,FL,33139.0,1750000.0,2.0,2.5,COSTA BRAVA CONDO,1733.0,NaN,1972.0,NaN,1010.0,2093.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/11-Islan...,MARMLS,A11708184,N,Y,25.789357,-80.147840,https://www.redfin.com/FL/Miami-Beach/11-Islan...,Stefania Cambarau,Douglas Elliman,David Siddons,Douglas Elliman,Yaron Lavi,Sterling Equity Realty LLC,None,None,7,blue,COSTA BRAVA CONDO at 11 Island Ave #1512 in Mi...
7,PAST SALE,2025-07-06,Condo/Co-op,300 Sunny Isle #808,Sunny Isles Beach,FL,33160.0,1500000.0,3.0,3.5,Parque Towers,1892.0,NaN,2019.0,NaN,

## Time on Market Calculator

In [70]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2025, 2, 24) ## List (Earlier) date
date2 = datetime(2025, 7, 3) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

129


## Clean CSV for Datawrapper Chart

In [71]:
chart_df = df_top_ten

In [72]:
chart_df = chart_df.fillna(" ")

In [73]:
chart_df['LISTING AGENTS'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING AGENTS'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [74]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [75]:
chart_df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME', 'LISTING AGENTS',
       'BUYING AGENTS'],
      dtype='object')

In [76]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "BEDS", 
    "BATHS",
    "SOURCE", 
    "MLS#"
]

In [77]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [78]:
chart_df = chart_df.rename(columns={'$/SQUARE FEET':'PPSF'})

In [79]:
chart_df['PPSF'].astype(float)

0    1866.0
1    1723.0
2    1200.0
3    1365.0
4    1114.0
5     585.0
6    1010.0
7     793.0
8     872.0
9     406.0
Name: PPSF, dtype: float64

In [80]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['PPSF'] = chart_df['PPSF'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))

In [81]:
desired_order = [
    'SOLD DATE',
    'LOCATION',
    'ADDRESS',
    'PRICE',
    'SQUARE FEET',
    'PPSF',
    'YEAR BUILT',
    'LISTING AGENTS',
    'BUYING AGENTS'
]

chart_df = chart_df[desired_order]


In [82]:
chart_df

,SOLD DATE,LOCATION,ADDRESS,PRICE,SQUARE FEET,PPSF,YEAR BUILT,LISTING AGENTS,BUYING AGENTS
0,2025-07-01,Grovenor House,2627 S Bayshore Dr #2304 in Coconut Grove,"$4,925,000","2,640","$1,866",2005,Lucas Boccheciampe Vantage Real Estate LLC,Debora Caridad Fortune Christie's Internationa...
1,2025-07-03,ISLAND VIEW ADDN,1201 20th St Ph 02 in Miami Beach,"$3,500,000","2,031","$1,723",2016,"Liz Hogan Compass Florida, LLC. Kaitlyn Neuman...","Liz Hogan Compass Florida, LLC. Kaitlyn Neuman..."
2,2025-07-03,BILTMORE PARC CONDO,718 Valencia Ave Ph 505 in Coral Gables,"$2,815,000","2,345","$1,200",2017,Anniella Tabraue One Sotheby's International R...,Tesalia Sacasa BHHS EWM Realty
3,2025-07-01,BALMORAL CONDO,9801 Collins Ave Unit 17S in Bal Harbour,"$2,650,000","1,942","$1,365",1977,Eva Bouhadana Beachfront Realty Inc Nicole Bou...,Jacob Assouline Luxuri International Real Esta...
4,2025-07-02,THE PINNACLE CONDO,17555 Collins Ave #2903 in Sunny Isles Beach,"$2,050,000","1,840","$1,114",1998,Ana Waxman Keller Williams Eagle Realty Evgeni...,Michael Lombard Coldwell Banker Realty
5,2025-07-02,SECOND WESTMORELAND ADD,1221 SW 12 Ct in Miami,"$1,815,000","3,103",$585,2025,Nancy Muniz Coldwell Banker Realty,Lina Barcelo Douglas Elliman
6,2025-07-01,COSTA BRAVA CONDO,11 Island Ave #1512 in Miami Beach,"$1,750,000","1,733","$1,010",1972,Stefania Cambarau Douglas Elliman David Siddon...,Yaron Lavi Sterling Equity Realty LLC
7,2025-07-06,Parque Towers,300 Sunny Isle #808 in Sunny Isles Beach,"$1,500,000","1,892",$793,2019,Lana Bell Douglas Elliman,Alena Siadletskaya Vera Realty LLC
8,2025-07-01,ONE PARAISO CONDO,3131 NE 7th Ave #706 in Miami,"$1,425,000","1,634",$872,2018,,
9,2025-07-01,HAMPTONS SOUTH CONDO,20201 E Country Club Dr #1502 in Aventura,"$1,325,000","3,262",$406,2004,,


In [83]:
csv_date_string = today.strftime("%m_%d_%Y")

In [84]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [85]:
chart_df.columns

Index(['SOLD DATE', 'LOCATION', 'ADDRESS', 'PRICE', 'SQUARE FEET', 'PPSF',
       'YEAR BUILT', 'LISTING AGENTS', 'BUYING AGENTS'],
      dtype='object')